In [1]:
IMAGE_DIMS = (50,34,50,1)
OUTPUT_DIR = '../../output/train-local/'
INPUT_DIR = '../../output/step5/'

In [2]:
import tflearn
from tflearn.layers.core import *
from tflearn.layers.conv import *
from tflearn.data_utils import *
from tflearn.layers.normalization import *
from tflearn.layers.estimator import regression
import os
import shutil
import datetime
import h5py

In [3]:
def log(message):
    print('{} {}'.format(datetime.datetime.now(), message))

In [4]:
def load_data(input_dir, image_dims):
    return h5py.File('{}data-centered-rotated-{}-{}-{}.h5'.format(input_dir,image_dims[2],image_dims[1],image_dims[0]), 'r')

In [5]:
def prepare_dirs(output_dir):
    log('Preparing output dir')

    dir_tflogs = output_dir + 'tf-logs'
    dir_checkpoints = output_dir + 'tf-checkpoints'
    dir_checkpoint_best = output_dir + 'tf-checkpoint-best'
    
#     shutil.rmtree(output_dir, True)
    try:
        os.makedirs(dir_tflogs)
        os.makedirs(dir_checkpoints)
        os.makedirs(dir_checkpoint_best)
    except:
        pass

    return dir_tflogs, dir_checkpoints, dir_checkpoint_best

In [6]:
def network(image_dims):
    net = input_data(shape=[None, image_dims[0], image_dims[1], image_dims[2], image_dims[3]], dtype=tf.float32)
    
    net = conv_3d(net, 32, 3, strides=1, activation='relu')
    net = max_pool_3d(net, [1,2,2,2,1], strides=[1,2,2,2,1])

    net = conv_3d(net, 64, 3, strides=1, activation='relu')
    net = max_pool_3d(net, [1,2,2,2,1], strides=[1,2,2,2,1])
    
    net = fully_connected(net, 64, activation='relu')
    net = dropout(net, 0.8)
    
    net = fully_connected(net, 2, activation='softmax')
    
    net = regression(net, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)
    return net

In [7]:
def prepare_cnn(output_dir, image_dims):
    dir_tflogs, dir_checkpoints, dir_checkpoint_best = prepare_dirs(output_dir)
    log('Preparing CNN network...')
    net = network(image_dims)

    log('Preparing DNN trainer...')
    model = tflearn.DNN(net, tensorboard_verbose=3, 
                             tensorboard_dir=dir_tflogs,
                             checkpoint_path=dir_checkpoints,
                             best_checkpoint_path=dir_checkpoint_best)
    return model

In [8]:
def start_training(model, input_dir, image_dims):

    with load_data(input_dir, image_dims) as hdf5:
        X = hdf5['X']
        Y = hdf5['Y']
        print('X shape', X.shape)
        print('Y shape', Y.shape)

        log('Starting CNN training...')
        model.fit(X, Y, validation_set=0.1, shuffle=True,
                  batch_size=10, n_epoch=10,
                  show_metric=True, 
                  run_id='first'+str(image_dims))

In [9]:
print('==== PREPARE CNN ====')
model = prepare_cnn(OUTPUT_DIR, IMAGE_DIMS)
print('==== DONE ====')

==== PREPARE CNN ====
2017-02-19 18:28:10.446595 Preparing output dir
2017-02-19 18:28:10.446842 Preparing CNN network...
2017-02-19 18:28:10.553738 Preparing DNN trainer...
==== DONE ====


In [10]:
print('==== TRAIN CNN ====')
start_training(model, INPUT_DIR, IMAGE_DIMS)
print('==== ALL DONE ====')

Training Step: 19  | total loss: 0.53220 | time: 41.943s
| Adam | epoch: 010 | loss: 0.53220 - acc: 0.7416 -- iter: 10/17
Training Step: 20  | total loss: 0.52014 | time: 75.302s
| Adam | epoch: 010 | loss: 0.52014 - acc: 0.6962 | val_loss: 1.82357 - val_acc: 0.5000 -- iter: 17/17
--
INFO:tensorflow:/root/output/train-local/tf-checkpoints-20 is not in all_model_checkpoint_paths. Manually adding it.
==== ALL DONE ====
